[Advent of Code - Day 20](https://adventofcode.com/2023/day/20)

# Import *Module Config*

In [1]:
import sys
from collections import defaultdict
from math import lcm

sys.path.insert(0, "../")
from utils import aoc_input as inp

config = inp.download_input(year="2023", day="20")

In [2]:
config[:5]

['%gv -> lq, pm',
 '%rv -> jd, nh',
 '%nh -> rs, jd',
 '&vt -> tj',
 '%zv -> pm, gv']

# Total *Low & High pulses*

In [3]:
BROADCASTER = "broadcaster"
FLIP_FLOP = "%"
CONJUNCTION = "&"

In [4]:
def reset_modules(config):
    modules = defaultdict(list)
    ff = dict()
    cnj = dict()

    for line in config:
        module, receive = line.split(" -> ")
        if module[0] == FLIP_FLOP:
            module = module[1:]
            ff[module] = 0
        elif module[0] == CONJUNCTION:
            module = module[1:]
            cnj[module] = dict()

        for r in receive.split(", "):
            modules[module].append(r)

    for c in cnj:
        for module, receive in modules.items():
            if c in receive:
                cnj[c][module] = 0

    return modules, ff, cnj

## Part 1: *Low & High pulses* multiplication

In [5]:
modules, ff, cnj = reset_modules(config)
times = 1_000
lows, highs = 0, 0

while times:
    times -= 1
    queue = [(BROADCASTER, False)]

    while queue:
        module, hi = queue.pop(0)
        lows += not hi
        highs += hi

        if module in ff:
            if hi:
                continue
            elif not hi and ff[module]:
                ff[module] = False
            elif not hi and not ff[module]:
                hi = not hi
                ff[module] = True
        elif module in cnj:
            if all(inp for inp in cnj[module].values()):
                hi = False
            else:
                hi = True

        for receive in modules[module]:
            queue.append((receive, hi))
            if receive in cnj:
                cnj[receive][module] = hi

lows * highs

818649769

## Part 2: Fewest *Button presses*

In [6]:
target = "rx"
modules, ff, cnj = reset_modules(config)
high_cycles = dict()

for module, receive in modules.items():
    if target in receive:
        for inp in cnj[module]:
            high_cycles[inp] = 0

times = 0

while any(inp == 0 for inp in high_cycles.values()):
    times += 1
    queue = [(BROADCASTER, False)]

    while queue:
        module, hi = queue.pop(0)

        if module in ff:
            if hi:
                continue
            elif not hi and ff[module]:
                ff[module] = False
            elif not hi and not ff[module]:
                hi = not hi
                ff[module] = True
        elif module in cnj:
            if all(inp for inp in cnj[module].values()):
                hi = False
            else:
                hi = True

        for receive in modules[module]:
            queue.append((receive, hi))
            if receive in cnj:
                cnj[receive][module] = hi

        if module in high_cycles and hi and not high_cycles[module]:
            high_cycles[module] = times

lcm(*high_cycles.values())

246313604784977